In [1]:
import os 
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), "..")) 
if project_root not in sys.path: 
    sys.path.insert(0, project_root)

project_root

'c:\\Users\\ray\\Documents\\2025 RA\\contrastive-learning-RL'

In [2]:
# Personal 
from src.data.TrajectorySet import TrajectorySet
from src.data.Sampler import Sampler 
from src.data.DatasetCL import DatasetCL
from src.models.cl_model import mlpCL
from src.trainers.cl_trainer import train_cl
from src.utils.visualizations import visualize_embeddings
from src.utils.tensor_utils import split_data

# Misc
import minari 

# Torch 
import torch 

# PyTorch Lightning 
import pytorch_lightning
from pytorch_lightning.loggers import WandbLogger

In [3]:
MINARI_DATASET = minari.load_dataset("D4RL/pointmaze/large-v2")
PROJECT_ROOT = os.getcwd() 
CHECKPOINT_PATH = PROJECT_ROOT + "/trained_models"

PROJECT_NAME = "Contrastive Learning RL"
RUN_NAME = "laplace_cos_sim"
FILENAME = RUN_NAME

CONFIG = {
        "distribution": "g",
        "num_state_pairs": 1_000_000,
        "batch_size": 4096,
        "lr": 1e-3,
        "weight_decay": 1e-5, 
        "temperature": 0.03796123348109251,
        "max_epochs": 1000,
        "filename": FILENAME,
        "device": "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
    }

In [4]:
CONFIG["device"]

'cuda'

In [5]:
wandb_logger = WandbLogger(
        project=PROJECT_NAME, 
        name=RUN_NAME, 
        save_dir = PROJECT_ROOT, 
        log_model=True,
        config = CONFIG) 

dist = CONFIG["distribution"]
num_state_pairs = CONFIG["num_state_pairs"]
batch_size = CONFIG["batch_size"]
lr = CONFIG["lr"]
weight_decay = CONFIG["weight_decay"]
temperature = CONFIG["temperature"]
max_epochs = CONFIG["max_epochs"]
device = CONFIG["device"]
filename = CONFIG["filename"]

In [6]:
T = TrajectorySet(dataset=MINARI_DATASET)
S = Sampler(T, dist=dist, b=15, sigma=15)

split_val = 0.8 
train_batch = int(round(num_state_pairs * split_val))
val_batch = int(round(num_state_pairs * (1 - split_val)))

train_dataset = DatasetCL(S, num_state_pairs=train_batch)
val_dataset = DatasetCL(S, num_state_pairs=val_batch)

print(train_batch)
print(val_batch)

800000
200000


In [7]:
MINIBATCH_SIZE = batch_size
print("Minibatch size:", MINIBATCH_SIZE)
print(type(MINIBATCH_SIZE))

Minibatch size: 4096
<class 'int'>


In [8]:
model = train_cl(cl_model=mlpCL, 
                train_ds=train_dataset, 
                val_ds = val_dataset,
                batch_size= MINIBATCH_SIZE,
                logger=wandb_logger, 
                checkpoint_path=CHECKPOINT_PATH,
                max_epochs=max_epochs,
                filename=filename,  
                device = device, 
                lr=lr, 
                temperature=temperature, 
                weight_decay = weight_decay,
                )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
c:\Users\ray\AppData\Local\anaconda3\envs\CL_RL_gpu\lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function 

c:\Users\ray\AppData\Local\anaconda3\envs\CL_RL_gpu\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:751: Checkpoint directory C:\Users\ray\Documents\2025 RA\contrastive-learning-RL\dev_notebooks\trained_models exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params | Mode 
--------------------------------------------
0 | mlp  | Sequential | 44.5 K | train
--------------------------------------------
44.5 K    Trainable params
0         Non-trainable params
44.5 K    Total params
0.178     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\ray\AppData\Local\anaconda3\envs\CL_RL_gpu\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


c:\Users\ray\AppData\Local\anaconda3\envs\CL_RL_gpu\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 999: 100%|██████████| 195/195 [00:09<00:00, 20.37it/s, v_num=p7cl, train/nll_loss_step=4.180, train/top1_step=0.188, train/top5_step=0.401, val/nll_loss=4.230, val/top1=0.177, val/top5=0.383, train/nll_loss_epoch=4.200, train/top1_epoch=0.178, train/top5_epoch=0.388] 

`Trainer.fit` stopped: `max_epochs=1000` reached.


Epoch 999: 100%|██████████| 195/195 [00:09<00:00, 20.36it/s, v_num=p7cl, train/nll_loss_step=4.180, train/top1_step=0.188, train/top5_step=0.401, val/nll_loss=4.230, val/top1=0.177, val/top5=0.383, train/nll_loss_epoch=4.200, train/top1_epoch=0.178, train/top5_epoch=0.388]
Best model path: C:\Users\ray\Documents\2025 RA\contrastive-learning-RL\dev_notebooks\trained_models\laplace_cos_sim-v1.ckpt
